In [1]:
import pandas as pd
import json
import yaml

In [2]:
def clean_data(df):
    for i,row in df.iterrows():
        df['info'][i]= yaml.load(df['info'][i])
    info = pd.DataFrame(list(df['info']))
    # Category Column
    info['categories'] = info['categories'].apply(lambda x: x[0]['title'])
    
    # Rating Column
    info['rating'] = info['rating'].str.replace('Decimal\(\'','').str.replace('\'\)','')
    info['rating'] = info['rating'].apply(lambda x: int(x))
    
    # Review Count 
    info['review_count'] = info['review_count'].str.replace('Decimal\(\'','').str.replace('\'\)','')
    info['review_count'] = info['review_count'].apply(lambda x: int(x))
    
    # Drop other columns
    info= info[['categories','rating','review_count']]
    df.drop('info',axis=1,inplace=True)
    df.drop('insertedAtTimestamp',axis=1,inplace=True)
    frames=[df,info]
    final = pd.concat([df,info], axis=1)
    return final

In [3]:
with open('yelp-restaurants.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
final = clean_data(df)
final.head()

,Id,categories,rating,review_count
0,uaFHoq-a5XqxF-bsOK9_Qg,Chinese,3,902
1,IhSVn0TaX8xXb3wcQ-fgcA,Chinese,4,2444
2,x9wa2M7gvf32swUSBP3uXA,Mexican,3,746
3,pmJqb7-EckluXUk6Rtml6A,Chinese,3,83
4,FZQ-l92qE1NqpI5lbmAM_g,Indian,3,11


In [4]:
final.to_csv('yelp.csv',index=False)

In [5]:
def create_testdata(final):
    fav = final.loc[(final['rating'] >= 4) & (final['review_count'] > 500)]
#     fav.sort_values('review_count', ascending=False, inplace=True)
    fav['recommended'] = 1
    
    hate = final.loc[(final['rating'] <= 3) & (final['review_count'] > 500)]
#     hate.sort_values('review_count', ascending=False, inplace=True)
    hate['recommended'] = 0
    
    frames = [fav[0:100],hate[0:100]]
    train = pd.concat(frames)
    
    return train

In [6]:
train = create_testdata(final)
idx1 = train.index
idx2 = final.index
idx3 = idx2.difference(idx1)
test = final.iloc[idx3,:]

/Users/maulikmodi/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/maulikmodi/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
train.to_csv('FILE_2.csv',index=False)
test.to_csv('FILE_1.csv',index=False)

In [8]:
# a = train.iloc[:,-1]
# a.shape
# a = train.iloc[:, train.columns != 'recommended']
# b = train.iloc[:,-1]
# train[train.columns[-1:]]
train_y = train.ix[:,-1]
train_X = train.ix[:,1:]

/Users/maulikmodi/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/Users/maulikmodi/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [9]:
train_y

1       1
11      1
18      1
20      1
24      1
29      1
45      1
64      1
66      1
67      1
74      1
88      1
95      1
105     1
107     1
124     1
133     1
151     1
158     1
180     1
187     1
195     1
202     1
206     1
208     1
214     1
219     1
225     1
226     1
231     1
       ..
1515    0
1533    0
1536    0
1539    0
1541    0
1589    0
1599    0
1600    0
1684    0
1694    0
1703    0
1717    0
1725    0
1752    0
1822    0
1823    0
1862    0
1943    0
1946    0
1958    0
1963    0
1967    0
1989    0
2003    0
2022    0
2039    0
2057    0
2078    0
2080    0
2082    0
Name: recommended, Length: 200, dtype: int64